<a href="https://colab.research.google.com/github/luisosmx/spark/blob/main/inegi_influx_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar -xzf spark-3.1.2-bin-hadoop3.2.tgz
!mv spark-3.1.2-bin-hadoop3.2.tgz /opt/spark-3.1.2
!pip install -q findspark
!pip install pyspark==3.1.2 
!ln -s /opt/spark-3.1.2 /opt/spark
!export SPARK_HOME=/opt/spark
!export PATH=$SPARK_HOME/bin:$PATH

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 KB 18.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=a7bce4ea14bc66f05f8e7cb8ae1c3a0dcb0a9ded9a429a58ea7f21cd295a5697
  Stored in directory: /root/.cache/pip/wheels/df/88/9e/58ef1f74892fef590330ca0830b5b6d995ba29b44f977b3926
Successfully built pyspark


In [2]:
!pip install pyspark==3.1.2 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip freeze

In [3]:
import os
import findspark

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop3.2'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.6.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'

findspark.init()

sc = pyspark.SparkContext("local[*]")
spark = SparkSession(sc)

print('Modules imported and Spark loaded')

Modules imported and Spark loaded


# Loading data into PySpark

Getting data from Github repo without clonning the project, just using [raw.githubusercontent.com](https://stackoverflow.com/questions/39065921/what-do-raw-githubusercontent-com-urls-represent)

In [4]:
!wget --continue /content/afluenciastc_simple_01_2023.csv

/content/afluenciastc_simple_01_2023.csv: Scheme missing.


Reading the file with a Spark dataframe

In [5]:
spark_df = spark.read.csv('/content/afluenciastc_simple_01_2023.csv')
spark_df.show(10)



+----------+----+-----+-------+-------------------+---------+
|       _c0| _c1|  _c2|    _c3|                _c4|      _c5|
+----------+----+-----+-------+-------------------+---------+
|     fecha|anio|  mes|  linea|           estacion|afluencia|
|2010-01-01|2010|Enero|Linea 1|           Zaragoza|    20227|
|2010-01-01|2010|Enero|Linea 1| Isabel la Católica|     6487|
|2010-01-01|2010|Enero|Linea 1|          Moctezuma|    10304|
|2010-01-01|2010|Enero|Linea 1|        Pino Suárez|     8679|
|2010-01-01|2010|Enero|Linea 1|       Gómez Farías|    19499|
|2010-01-01|2010|Enero|Linea 6|Deptvo. 18 de Marzo|      621|
|2010-01-01|2010|Enero|Linea 6|  La Villa-Basilica|    24792|
|2010-01-01|2010|Enero|Linea 9|          Pantitlán|    27000|
|2010-01-01|2010|Enero|Linea 8|             Aculco|     3652|
+----------+----+-----+-------+-------------------+---------+
only showing top 10 rows



In [6]:
df = spark.read.option("header",True).csv('/content/afluenciastc_simple_01_2023.csv')
df.show(20)

+----------+----+-----+-------+-------------------+---------+
|     fecha|anio|  mes|  linea|           estacion|afluencia|
+----------+----+-----+-------+-------------------+---------+
|2010-01-01|2010|Enero|Linea 1|           Zaragoza|    20227|
|2010-01-01|2010|Enero|Linea 1| Isabel la Católica|     6487|
|2010-01-01|2010|Enero|Linea 1|          Moctezuma|    10304|
|2010-01-01|2010|Enero|Linea 1|        Pino Suárez|     8679|
|2010-01-01|2010|Enero|Linea 1|       Gómez Farías|    19499|
|2010-01-01|2010|Enero|Linea 6|Deptvo. 18 de Marzo|      621|
|2010-01-01|2010|Enero|Linea 6|  La Villa-Basilica|    24792|
|2010-01-01|2010|Enero|Linea 9|          Pantitlán|    27000|
|2010-01-01|2010|Enero|Linea 8|             Aculco|     3652|
|2010-01-01|2010|Enero|Linea 9|          Velódromo|     3239|
|2010-01-01|2010|Enero|Linea 5|Autobuses del Norte|    16824|
|2010-01-01|2010|Enero|Linea 5|          Misterios|     3513|
|2010-01-01|2010|Enero|Linea 7|     Constituyentes|     1417|
|2010-01

In [7]:
spark.read.option("header",True) \
          .csv("/content/afluenciastc_simple_01_2023.csv") \
          .createOrReplaceTempView("influx_file")

In [8]:
spark.sql("SELECT * FROM influx_file").show(10)

+----------+----+-----+-------+-------------------+---------+
|     fecha|anio|  mes|  linea|           estacion|afluencia|
+----------+----+-----+-------+-------------------+---------+
|2010-01-01|2010|Enero|Linea 1|           Zaragoza|    20227|
|2010-01-01|2010|Enero|Linea 1| Isabel la Católica|     6487|
|2010-01-01|2010|Enero|Linea 1|          Moctezuma|    10304|
|2010-01-01|2010|Enero|Linea 1|        Pino Suárez|     8679|
|2010-01-01|2010|Enero|Linea 1|       Gómez Farías|    19499|
|2010-01-01|2010|Enero|Linea 6|Deptvo. 18 de Marzo|      621|
|2010-01-01|2010|Enero|Linea 6|  La Villa-Basilica|    24792|
|2010-01-01|2010|Enero|Linea 9|          Pantitlán|    27000|
|2010-01-01|2010|Enero|Linea 8|             Aculco|     3652|
|2010-01-01|2010|Enero|Linea 9|          Velódromo|     3239|
+----------+----+-----+-------+-------------------+---------+
only showing top 10 rows



In [12]:
df_1 = spark.sql("""
  SELECT 
    fecha AS date
    , anio AS year
    , mes AS month
    , linea AS line
    , estacion AS station
    , afluencia AS influx
  FROM influx_file
""")

#Replace months by numbers

In [15]:
df_1.createOrReplaceTempView("df_1")

In [18]:
spark.sql("""
  SELECT
      CASE 
        WHEN month = 'Enero' THEN '01'
        WHEN month = 'Febrero' THEN '02'
        WHEN month = 'Marzo' THEN '03'
        WHEN month = 'Abril' THEN '04'
        WHEN month = 'Mayo' THEN '05'
        WHEN month = 'Junio' THEN '06'
        WHEN month = 'Julio' THEN '07'
        WHEN month = 'Agosto' THEN '08'
        WHEN month = 'Septiembre' THEN '09'
        WHEN month = 'Octubre' THEN '10'
        WHEN month = 'Noviembre' THEN '11'
        WHEN month = 'Diciembre' THEN '12'
        ELSE month
      END AS Month
  FROM df_1
""")

+-----+
|Month|
+-----+
|   01|
|   01|
|   01|
|   01|
|   01|
|   01|
|   01|
|   01|
|   01|
|   01|
+-----+
only showing top 10 rows



#Replace line by numbers

#Sorting